In [2]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
ie_data = pd.read_csv("../data/ie_data_tax.csv", index_col=False)
ie_data = ie_data[ie_data["Date"]<2011.09]
print(ie_data.columns)
ie_data.head()

In [ ]:
preprocess = lambda x: float(x.replace(" ", "").replace(",", "")) if not isinstance(x, float) else x
ie_data["CAPE"] = ie_data["CAPE"].apply(preprocess)
ie_data["Real_Total_Return_Price"] = ie_data["Real_Total_Return_Price"].apply(preprocess)

In [3]:
ie_data = pd.read_excel("../data/ie_data.xls", sheet_name="Data", skiprows=7)
ie_data = ie_data[["Date", "CAPE", "Yield", "Real Return", "Real Return.1", "Returns.2"]].copy()
ie_data.rename(columns={"Yield": "Excess_CAPE_Yield", "Real Return": "Real_Returns_Equity", "Real Return.1": "Real_Returns_Bond", "Returns.2": "Excess_Real_Returns_Equity"}, inplace=True)
# Drop NAs, sample period: 1881.01-2011.08
ie_data = ie_data.dropna()
ie_data.head()

,Date,CAPE,Excess_CAPE_Yield,Real_Returns_Equity,Real_Returns_Bond,Excess_Real_Returns_Equity
120,1881.01,18.473952,-0.010489,0.045353,0.056468,-0.011115
121,1881.02,18.147258,-0.011393,0.046774,0.056199,-0.009425
122,1881.03,18.270119,-0.013123,0.042423,0.054885,-0.012462
123,1881.04,17.950108,-0.007504,0.045971,0.054635,-0.008665
124,1881.05,18.869719,-0.008881,0.041157,0.054786,-0.013628


In [ ]:
x = 1/(np.array(ie_data[(ie_data["Date"]<=2006)]["CAPE"]).astype(float))
x = sm.add_constant(x)
y = np.array(ie_data[(ie_data["Date"]<=2006)]["Real_Returns_Equity"]).astype(float)

model = sm.OLS(y, x, missing="drop")
summary = model.fit()
summary.summary()

In [ ]:
def cape_regression(cape_values, time_period, data=ie_data):
    x = np.array(cape_values)
    x = sm.add_constant(x)
    model = sm.OLS(np.array(data["Real_Total_Return_Price"]), x, missing="drop")
    summary = model.fit()